# 2.3. [**Partitionnement**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/2_3_clustering.ipynb)</br>([*Clustering*](https://scikit-learn.org/stable/modules/clustering.html))

Le [**partitionnement de données**](https://en.wikipedia.org/wiki/Cluster_analysis) (*clustering*) non étiquetées peut être effectué avec le module [**`sklearn.cluster`**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster).

Chaque algorithme de clustering se décline en deux variantes : une classe, qui implémente la méthode `fit` pour découvrir les clusters sur les données d'entraînement, et une fonction, qui, compte tenu des données d'entraînement, renvoie un tableau d'étiquettes entières correspondant aux différents clusters. Pour la classe, les étiquettes sur les données d'apprentissage se trouvent dans l'attribut `labels_`.

**Données d'entrée**

Une chose importante à noter est que les algorithmes implémentés dans ce module peuvent prendre différents types de matrices en entrée. Toutes les méthodes acceptent des matrices de données standard de forme `(n_samples, n_features)`. Celles-ci peuvent être obtenues à partir des classes du module [**`sklearn.feature_extraction`**](https://scikit-learn.org/stable/modules/clustering.html). Pour [**`AffinityPropagation`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html), [**`SpectralClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html) et [**`DBSCAN`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html), on peut également entrer des matrices de similarité de forme `(n_samples, n_samples)`. Celles-ci peuvent être obtenues à partir des fonctions du module [**`sklearn.metrics.pairwise`**](https://scikit-learn.org/stable/modules/classes.html).

## 2.3.1. **Vue d'ensemble des méthodes de partitionnement**<br/>([*Overview of clustering methods*](https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods))

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png)

|Nom de la méthode|Paramètres|Évolutivité|Cas d'utilisation|Géométrie (métrique utilisée)|
|-|-|-|-|-|
|[**K-Means** (2.3.2)](https://scikit-learn.org/stable/modules/clustering.html#k-means)|nombre de clusters|`n_samples` très grand, `n_clusters` moyen avec [**code MiniBatch** (2.3.2.2)](https://scikit-learn.org/stable/modules/clustering.html#mini-batch-kmeans)|Usage général, même taille de cluster, géométrie plate, pas trop de clusters, inductif|Distances entre les points|
|[**Propagation par affinité** (2.3.3)](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation)|amortissement, préférence d'échantillon|Non évolutif avec `n_samples`|Nombreux clusters, taille de cluster inégale, géométrie non plate, inductif|Distance du graphe (par exemple, graphe du plus proche voisin)|
|[**Déplacement moyen** (2.3.4)](https://scikit-learn.org/stable/modules/clustering.html#mean-shift)|bande passante|Non évolutif avec `n_samples`|De nombreux clusters, taille de cluster inégale, géométrie non plate, inductif|Distances entre les points|
|[**Regroupement spectral** (2.3.5)](https://scikit-learn.org/stable/modules/clustering.html#spectral-clustering)|nombre de clusters|`n_samples` moyen, petit `n_clusters`|Peu de clusters, même taille de cluster, géométrie non plate, transductif|Distance du graphe (par exemple, graphe du plus proche voisin)|
|[**Regroupement hiérarchique** (2.3.6)](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)|nombre de clusters ou seuil de distance|Grands `n_samples` et `n_clusters`|De nombreux clusters, éventuellement des contraintes de connectivité, transductif|Distances entre les points|
|[**Partitionnement agglomératif** (2.3.6)](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)|nombre de clusters ou seuil de distance, type de liaison, distance|Grands `n_samples` et `n_clusters`|De nombreux clusters, éventuellement des contraintes de connectivité, des distances non euclidiennes, transductif|Toute distance par paire|
|[**DBSCAN** (2.3.7)](https://scikit-learn.org/stable/modules/clustering.html#dbscan)|taille du voisinage|`n_samples` très grand, `n_clusters` moyen|Géométrie non plate, tailles de cluster inégales, suppression des valeurs aberrantes, transductif|Distances entre les points les plus proches|
|[**OPTICS** (2.3.8)](https://scikit-learn.org/stable/modules/clustering.html#optics)|appartenance minimale au cluster|Très grand `n_samples`, grand `n_clusters`|Géométrie non plate, tailles de cluster inégales, densité de cluster variable, suppression des valeurs aberrantes, transductif|Distances entre les points|
|[**Mélanges gaussiens** (2.1)](https://scikit-learn.org/stable/modules/mixture.html#mixture)|beaucoup|Non évolutif|Géométrie plate, bonne pour l'estimation de la densité, inductif|Distances de Mahalanobis aux centres|
|[**BIRCH** (2.3.9)](https://scikit-learn.org/stable/modules/clustering.html#birch)|facteur de ramification, seuil, clustereur global facultatif|Grands `n_clusters` et `n_samples`|Grand ensemble de données, suppression des valeurs aberrantes, réduction des données, inductif|Distance euclidienne entre les points|
|[**Bisecting K-Means** (2.3.6.5)](https://scikit-learn.org/stable/modules/clustering.html#bisect-k-means)|nombre de clusters|`n_samples` très grand, `n_clusters` moyen|Usage général, même taille de cluster, géométrie plate, pas de clusters vides, inductif, hiérarchique|Distances entre les points|

Le clustering de géométrie non plane est utile lorsque les clusters ont une forme spécifique, c'est-à-dire une variété non plate, et que la distance euclidienne standard n'est pas la bonne métrique. Ce cas se présente dans les deux rangées supérieures de la figure ci-dessus.

Les modèles de mélange gaussiens, utiles pour le clustering, sont décrits dans un [**autre chapitre de la documentation dédié aux modèles de mélange** (2.1)](https://scikit-learn.org/stable/modules/mixture.html#mixture). KMeans peut être considéré comme un cas particulier de modèle de mélange gaussien avec une covariance égale par composant.

Les méthodes de regroupement [**transductives**](https://scikit-learn.org/stable/glossary.html#term-transductive) (contrairement aux méthodes de regroupement [**inductives**](https://scikit-learn.org/stable/glossary.html#term-inductive)) ne sont pas conçues pour être appliquées à de nouvelles données invisibles.

## 2.3.2. **K-moyennes**<br/>([*K-means*](https://scikit-learn.org/stable/modules/clustering.html#k-means))

L'algorithme [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) regroupe les données en essayant de séparer les échantillons en $n$ groupes de variance égale, en minimisant un critère connu sous le nom d'inertie ou somme des carrés intra-cluster (voir ci-dessous). Cet algorithme nécessite que le nombre de clusters soit spécifié. Il s'adapte bien à un grand nombre d'échantillons et a été utilisé dans un large éventail de domaines d'application dans de nombreux domaines.

L'algorithme des k-moyennes divise un ensemble de $N$ échantillons en $K$ grappes $C$ disjointes, chacune décrite par la moyenne $\mu_j$ des échantillons de la grappe. Les moyennes sont communément appelées les "barycentres" (*centroïdes*) du cluster; noter qu'elles ne sont pas, en général, des points de $X$, bien qu'elles partagent le même espace.

L'algorithme K-means vise à choisir des centroïdes qui minimisent l'**inertie**, ou **critère de somme des carrés intra-cluster** :

$$\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)$$

L'inertie peut être reconnue comme une mesure de la cohérence interne des clusters. Elle souffre de divers inconvénients :

L'inertie peut être reconnue comme une mesure de la cohérence interne des clusters. Il souffre de divers inconvénients :
* L'inertie fait l'hypothèse que les clusters sont convexes et isotropes, ce qui n'est pas toujours le cas. Il répond mal aux grappes allongées ou aux variétés aux formes irrégulières.
* L'inertie n'est pas une métrique normalisée : nous savons simplement que les valeurs inférieures sont meilleures et que zéro est optimal. Mais dans les espaces de très grande dimension, les distances euclidiennes ont tendance à gonfler (c'est un exemple de ce qu'on appelle la "malédiction de la dimensionnalité"). L'exécution d'un algorithme de réduction de dimensionnalité tel que l'[**analyse en composantes principales (ACP)** (2.5.1)](https://scikit-learn.org/stable/modules/decomposition.html#pca) avant le clustering k-means peut atténuer ce problème et accélérer les calculs.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_assumptions_001.png)

Le K-means est souvent appelé algorithme de Lloyd. En termes simples, l'algorithme comporte trois étapes. La première étape consiste à choisir les centroïdes initiaux, la méthode la plus élémentaire étant de choisir $k$ échantillons dans l'ensemble de données $X$. Après initialisation, K-means consiste à boucler entre les deux autres étapes. La première étape affecte chaque échantillon à son centroïde le plus proche. La deuxième étape crée de nouveaux centroïdes en prenant la valeur moyenne de tous les échantillons attribués à chaque centroïde précédent. La différence entre l'ancien et le nouveau barycentre est calculée et l'algorithme répète ces deux dernières étapes jusqu'à ce que cette valeur soit inférieure à un seuil. En d'autres termes, il se répète jusqu'à ce que les centroïdes ne bougent pas de manière significative.

K-means équivaut à l'algorithme de maximisation des attentes avec une petite matrice de covariance diagonale entièrement égale.

L'algorithme peut également être compris à travers le concept de [**diagrammes de Voronoi** (wkpd)](https://en.wikipedia.org/wiki/Voronoi_diagram). Tout d'abord, le diagramme de Voronoi des points est calculé à l'aide des centroïdes actuels. Chaque segment du diagramme de Voronoi devient un cluster distinct. Deuxièmement, les centroïdes sont mis à jour à la moyenne de chaque segment. L'algorithme répète ensuite ceci jusqu'à ce qu'un critère d'arrêt soit rempli. Habituellement, l'algorithme s'arrête lorsque la diminution relative de la fonction objectif entre les itérations est inférieure à la valeur de tolérance donnée. Ce n'est pas le cas dans cette implémentation : l'itération s'arrête lorsque les centroïdes se déplacent moins que la tolérance.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_digits_001.png)

Avec suffisamment de temps, les K-moyennes convergeront toujours, mais cela peut être à un minimum local. Cela dépend fortement de l'initialisation des centroïdes. De ce fait, le calcul est souvent effectué plusieurs fois, avec des initialisations différentes des barycentres. Une méthode pour aider à résoudre ce problème est le schéma d'initialisation k-means++, qui a été implémenté dans scikit-learn (utilisez le paramètre `init='k-means++'`). Cela initialise les centroïdes pour qu'ils soient (généralement) distants les uns des autres, ce qui donne probablement de meilleurs résultats que l'initialisation aléatoire, comme indiqué dans la référence.

K-means++ peut également être appelé indépendamment pour sélectionner des graines pour d'autres algorithmes de clustering, voir [**`sklearn.cluster.kmeans_plusplus`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.kmeans_plusplus.html) pour plus de détails et un exemple d'utilisation.

L'algorithme prend en charge les poids d'échantillon, qui peuvent être donnés par un paramètre `sample_weight`. Cela permet d'attribuer plus de poids à certains échantillons lors du calcul des centres de cluster et des valeurs d'inertie. Par exemple, attribuer un poids de 2 à un échantillon équivaut à ajouter un doublon de cet échantillon à l'ensemble de données $X$.

K-means peut être utilisé pour la quantification vectorielle. Ceci est réalisé en utilisant la méthode de transformation d'un modèle entraîné de [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

### 2.3.2.1 **Parallélisme de bas niveau**<br/>([*Low-level parallelism*](https://scikit-learn.org/stable/modules/clustering.html#low-level-parallelism))

[**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) bénéficie du parallélisme basé sur OpenMP via Cython. De petits morceaux de données (256 échantillons) sont traités en parallèle, ce qui en plus produit une faible empreinte mémoire. Pour plus de détails sur la façon de contrôler le nombre de threads, veuillez vous référer à nos notes sur le [**parallélisme** (8.3.1)](https://scikit-learn.org/stable/computing/parallelism.html#parallelism).

#### Exemples

##### [**Démonstration des hypothèses de k-means**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_kmeans_assumptions.ipynb)<br/>([*Demonstration of k-means assumptions*](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html))

Démontrer quand k-means fonctionne intuitivement et quand ce n'est pas le cas.

##### [**Une démonstration du clustering K-Means sur les données des chiffres manuscrits**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_kmeans_digits.ipynb)<br/>([*A demo of K-Means clustering on the handwritten digits data*](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html))

Clustering des chiffres manuscrits.

#### Références

[?] Arthur, David and Sergei Vassilvitskii [“**k-means++: The advantages of careful seeding**](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf)[”](https://drive.google.com/file/d/11vuxCwPrrjuv8JakiEF5TCYyMBZzqoVg/view?usp=share_link), Actes du dix-huitième symposium annuel ACM-SIAM sur les algorithmes discrets, Society for Industrial and Applied Mathematics (2007)

### 2.3.2.2. **Mini lot K-moyennes**<br/>([Mini Batch K-Means](https://scikit-learn.org/stable/modules/clustering.html#mini-batch-k-means))

Le [**`MiniBatchKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) est une variante de l'algorithme [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) qui utilise des mini-batchs pour réduire le temps de calcul, tout en essayant d'optimiser la même fonction objectif. Les mini-lots sont des sous-ensembles des données d'entrée, échantillonnés de manière aléatoire à chaque itération d'apprentissage. Ces mini-lots réduisent considérablement la quantité de calculs nécessaires pour converger vers une solution locale. Contrairement à d'autres algorithmes qui réduisent le temps de convergence des k-means, les mini-batch k-means produisent des résultats qui ne sont généralement que légèrement moins bons que l'algorithme standard.

L'algorithme itère entre deux étapes principales, similaires au k-means vanille ??. Dans la première étape, $b$ échantillons sont tirés au hasard de l'ensemble de données, pour former un mini-lot. Ceux-ci sont ensuite affectés au centroïde le plus proche. Dans la deuxième étape, les centroïdes sont mis à jour. Contrairement aux k-moyennes, cela se fait sur une base par échantillon. Pour chaque échantillon du mini-lot, le centroïde attribué est mis à jour en prenant la moyenne en continu de l'échantillon et de tous les échantillons précédents attribués à ce centroïde. Cela a pour effet de diminuer le taux de variation d'un centroïde au fil du temps. Ces étapes sont exécutées jusqu'à ce qu'une convergence ou un nombre prédéterminé d'itérations soit atteint.

[**`MiniBatchKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) converge plus rapidement que [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), mais la qualité des résultats est réduite. En pratique, cette différence de qualité peut être assez faible, comme le montrent l'exemple et la référence citée.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_mini_batch_kmeans_001.png)

#### Exemples

##### [**Comparaison des algorithmes de clustering K-Means et MiniBatchKMeans**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_mini_batch_kmeans.ipynb)<br/>([*Comparison of the K-Means and MiniBatchKMeans clustering algorithms*](https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html))

Comparaison de KMeans et MiniBatchKMeans.

##### [**Regroupement de documents texte à l'aide de k-means**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/text/plot_document_clustering.ipynb)<br/>([*Clustering text documents using k-means*](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html))

Regroupement de documents à l'aide de MiniBatchKMeans creux.

##### [**Apprentissage en ligne d'un dictionnaire de parties de visages**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_dict_face_patches.ipynb)<br/>([*Online learning of a dictionary of parts of faces*](https://scikit-learn.org/stable/auto_examples/cluster/plot_dict_face_patches.html))


#### Références

[?] D. Sculley [“**Web Scale K-Means clustering**](https://www.eecs.tufts.edu/~dsculley/papers/fastkmeans.pdf)[”](https://drive.google.com/file/d/1urEewuktSLJpV_PhQh7ita5O-VjRLtyw/view?usp=share_link), Proceedings of the 19th international conference on World wide web (2010)



## 2.3.3. **Propagation par affinité**<br/>([*Affinity Propagation*](https://scikit-learn.org/stable/modules/clustering.html#adjusted-rand-score))

[**`AffinityPropagation`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html) crée des clusters en envoyant des messages entre des paires d'échantillons jusqu'à convergence. Un ensemble de données est ensuite décrit à l'aide d'un petit nombre d'exemplaires, qui sont identifiés comme étant les plus représentatifs des autres échantillons. Les messages envoyés entre les paires représentent l'aptitude d'un échantillon à être l'exemplaire de l'autre, qui est mis à jour en réponse aux valeurs des autres paires. Cette mise à jour se produit de manière itérative jusqu'à la convergence, moment auquel les exemplaires finaux sont choisis, et donc le regroupement final est donné.

<figure class="align-center">
<a class="reference external image-reference" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_affinity_propagation.html"><img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_affinity_propagation_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_affinity_propagation_001.png" style="width: 320.0px; height: 240.0px;" /></a>
</figure>

La propagation par affinité peut être intéressante car elle choisit le nombre de clusters en fonction des données fournies. A cet effet, les deux paramètres importants sont la préférence, qui contrôle le nombre d'exemplaires utilisés, et le facteur d'amortissement qui amortit les messages de responsabilité et de disponibilité pour éviter les oscillations numériques lors de la mise à jour de ces messages.

Le principal inconvénient de la propagation par affinité est sa complexité. L'algorithme a une complexité temporelle de l'ordre de $\mathcal{O}(n^2 T)$, où $n$ est le nombre d'échantillons et $T$ est le nombre d'itérations jusqu'à la convergence. De plus, la complexité mémoire est de l'ordre de $\mathcal{O}(n^2)$ si une matrice de similarité dense est utilisée, mais réductible si une matrice de similarité creuse est utilisée. Cela rend la propagation par affinité plus appropriée pour les ensembles de données de petite à moyenne taille.

### Exemples

#### [**Démo de l'algorithme de clustering par propagation d'affinité**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_affinity_propagation.ipynb)<br/>([*Demo of affinity propagation clustering algorithm*](https://scikit-learn.org/stable/auto_examples/cluster/plot_affinity_propagation.html))

Propagation d'affinité sur un jeu de données 2D synthétique avec 3 classes.

#### [**Visualisation de la structure de la bourse**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/applications/plot_stock_market.ipynb)<br/>([*Visualizing the stock market structure*](https://scikit-learn.org/stable/auto_examples/applications/plot_stock_market.html))

Propagation d'affinité sur les séries chronologiques financières pour trouver des groupes d'entreprises.

### Description de l'algorithme

Les messages envoyés entre les points appartiennent à l'une des deux catégories. Le premier est la responsabilité $r(i, k)$, qui est la preuve accumulée que l'échantillon $k$ devrait être le représentant de l'échantillon $i$. La seconde est la disponibilité $a(i, k)$ qui est la preuve accumulée que l'échantillon $i$ devrait choisir l'échantillon $k$ comme son représentant, et considère les valeurs de tous les autres échantillons dont $k$ devrait être le représentant. De cette manière, les représentants sont choisis par des échantillons s'ils sont (1) suffisamment similaires à de nombreux échantillons et (2) choisis par de nombreux échantillons pour être représentatifs d'eux-mêmes.

Plus formellement, la responsabilité d'un échantillon $k$ pour être le représentant de l'échantillon $i$ est donné par :

$$r(i, k) \leftarrow s(i, k) - max [ a(i, k') + s(i, k') \forall k' \neq k ]$$

Où $s(i, k)$ est la similarité entre les échantillons $i$ et $k$. La disponibilité de l'échantillon $k$ pour être le représentant de l'échantillon $i$ est donné par:

$$a(i, k) \leftarrow min [0, r(k, k) + \sum_{i'~s.t.~i' \notin \{i, k\}}{r(i', k)}]$$

Pour commencer, toutes les valeurs de $r$ et $a$ sont mises à zéro, et le calcul de chacun itère jusqu'à convergence. Comme discuté ci-dessus, afin d'éviter les oscillations numériques lors de la mise à jour des messages, le facteur d'amortissement $\lambda$ est introduit au processus d'itération :

$$r_{t+1}(i, k) = \lambda\cdot r_{t}(i, k) + (1-\lambda)\cdot r_{t+1}(i, k)$$
$$a_{t+1}(i, k) = \lambda\cdot a_{t}(i, k) + (1-\lambda)\cdot a_{t+1}(i, k)$$

où $t$ indique les temps d'itération.

## 2.3.4. Décalage moyen

Le clustering [**`MeanShift`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html) vise à découvrir des *blobs* dans une densité lisse d'échantillons. Il s'agit d'un algorithme basé sur le centroïde, qui fonctionne en mettant à jour les candidats pour que les centroïdes soient la moyenne des points dans une région donnée. Ces candidats sont ensuite filtrés dans une étape de post-traitement pour éliminer les quasi-doublons pour former l'ensemble final de centroïdes.

Étant donné un barycentre candidat $x_i$ pour l'itération $t$, le candidat est mis à jour selon l'équation suivante :

$$x_i^{t+1} = m(x_i^t)$$

Où $N(x_i)$ est le voisinage des échantillons à une distance donnée autour de $x_i$ et $m$ est le vecteur de *décalage moyen* qui est calculé pour chaque centroïde qui pointe vers une région d'augmentation maximale de la densité de points. Ceci est calculé à l'aide de l'équation suivante, mettant à jour efficacement un centroïde pour qu'il soit la moyenne des échantillons dans son voisinage :

$$m(x_i) = \frac{\sum_{x_j \in N(x_i)}K(x_j - x_i)x_j}{\sum_{x_j \in N(x_i)}K(x_j - x_i)}$$

L'algorithme définit automatiquement le nombre de clusters, au lieu de s'appuyer sur le paramètre de bande passante `bandwidth`, qui dicte la taille de la région à parcourir. Ce paramètre peut être défini manuellement, mais peut être estimé à l'aide de la fonction `estimate_bandwidth` fournie, qui est appelée si la bande passante n'est pas définie.

L'algorithme n'est pas hautement évolutif (passage à l'échelle), car il nécessite plusieurs recherches du plus proche voisin pendant son exécution. L'algorithme converge nécessairement, mais il cessera d'itérer lorsque le changement de centroïdes est faible.

L'étiquetage d'un nouvel échantillon est effectué en trouvant le centroïde le plus proche pour un échantillon donné.

<figure class="align-center">
<a class="reference external image-reference" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_mean_shift.html"><img alt="../_images/sphx_glr_plot_mean_shift_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_mean_shift_001.png" style="width: 320.0px; height: 240.0px;" /></a>
</figure>

### Exemples

#### [**Une démonstration de l'algorithme de clustering par décalage moyen**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_mean_shift.ipynb)<br/>([*A demo of the mean-shift clustering algorithm*](https://scikit-learn.org/stable/auto_examples/cluster/plot_mean_shift.html))

Clustering par décalage moyen sur un jeu de données 2D synthétique avec 3 classes.

### Références

[?] Dorin Comaniciu, Peter Meer, [“**Mean Shift: A robust approach toward feature space analysis**](https://ieeexplore.ieee.org/document/1000236)[”](https://drive.google.com/file/d/1v5NOqj-DIjEZDWrZfN2yIRw_vbwMu0Ea/view?usp=share_link). IEEE Transactions on Pattern Analysis and Machine Intelligence. 2002. pp. 603-619.

## 2.3.5. Regroupement spectral

[**`SpectralClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html) effectue une intégration de faible dimension de la matrice d'affinité entre les échantillons, suivie d'un regroupement, par exemple, par KMeans, des composants des vecteurs propres dans l'espace de faible dimension. Il est particulièrement efficace en termes de calcul si la matrice d'affinité est creuse et que le solveur `amg` est utilisé pour le problème des valeurs propres (notez que le solveur `amg` nécessite que le module [**`pyamg`**](https://github.com/pyamg/pyamg) soit installé.)

La version actuelle de SpectralClustering nécessite que le nombre de clusters soit spécifié à l'avance. Cela fonctionne bien pour un petit nombre de clusters, mais n'est pas conseillé pour de nombreux clusters.

Pour deux clusters, SpectralClustering résout une relaxation convexe du problème des [coupes normalisées](https://people.eecs.berkeley.edu/~malik/papers/SM-ncut.pdf) sur le graphe de similarité : coupant le graphe en deux pour que le poids des arêtes coupées soit petit par rapport aux poids des arêtes à l'intérieur de chaque cluster. Ce critère est particulièrement intéressant lorsque l'on travaille sur des images, où les sommets du graphe sont des pixels, et les poids des arêtes du graphe de similarité sont calculés en fonction d'un gradient de l'image.

<img alt="noisy_img" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_segmentation_toy_001.png" style="width: 500.0px; height: 250.0px;" /></a> <a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_segmentation_toy.html"><img alt="segmented_img" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_segmentation_toy_002.png" style="width: 500.0px; height: 250.0px;" />


### Avertissement

Transformer la distance en similitudes bien élevées

Notez que si les valeurs de votre matrice de similarité ne sont pas bien distribuées, par ex. avec des valeurs négatives ou avec une matrice de distance plutôt qu'une similarité, le problème spectral sera singulier et le problème non résoluble. Dans ce cas il est conseillé d'appliquer une transformation sur les entrées de la matrice. Par exemple, dans le cas d'une matrice de distance signée, il est courant d'appliquer un noyau de chaleur :

    similarité = np.exp(-beta * distance / distance.std())

Voir les exemples d'une telle application.

### Exemples

#### [**Regroupement spectral pour la segmentation d'images**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_segmentation_toy.ipynb)<br/>([*Spectral clustering for image segmentation*](https://scikit-learn.org/stable/auto_examples/cluster/plot_segmentation_toy.html))

Segmentation d'objets à partir d'un arrière-plan bruyant à l'aide du regroupement spectral.

#### [**Segmenter en régions l'image des pièces de monnaie grecques**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_coin_segmentation.ipynb)<br/>([*Segmenting the picture of greek coins in regions*](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html))

Regroupement spectral pour diviser l'image des pièces de monnaie en régions.

#### 2.3.5.1. Différentes stratégies d'attribution d'étiquettes

Différentes stratégies d'attribution d'étiquettes peuvent être utilisées, correspondant au paramètre `assign_labels` de [**`SpectralClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html). La stratégie `"kmeans"` peut correspondre à des détails plus fins, mais peut être instable. En particulier, à moins que vous ne contrôliez le `random_state`, il peut ne pas être reproductible d'une exécution à l'autre, car il dépend de l'initialisation aléatoire. La stratégie alternative de `"discretize"` est reproductible à 100%, mais tend à créer des parcelles de forme assez régulière et géométrique. L'option `"cluster_qr"` récemment ajoutée est une alternative déterministe qui tend à créer le meilleur partitionnement visuel sur l'exemple d'application ci-dessous.

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p><code class="docutils literal notranslate"><span class="pre">assign_labels=&quot;kmeans&quot;</span></code></p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">assign_labels=&quot;discretize&quot;</span></code></p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">assign_labels=&quot;cluster_qr&quot;</span></code></p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html"><img alt="coin_kmeans" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_segmentation_001.png" style="width: 175.0px; height: 175.0px;" /></a></p></td>
<td><p><a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html"><img alt="coin_discretize" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_segmentation_002.png" style="width: 175.0px; height: 175.0px;" /></a></p></td>
<td><p><a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html"><img alt="coin_cluster_qr" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_segmentation_003.png" style="width: 175.0px; height: 175.0px;" /></a></p></td>
</tr>
</tbody>
</table>

##### Références

[?] Stella X. Yu, Jianbo Shi, [“**Multiclass spectral clustering**](http://lear.inrialpes.fr/people/triggs/events/iccv03/cdrom/iccv03/0313_yu.pdf)[”](https://drive.google.com/file/d/1JsQ_aa3BheGcipQONMcdZa0AzISuGJ-C/view?usp=share_link), 2003

[?] Anil Damle, Victor Minden, Lexing Ying, [“**Simple, direct, and efficient multi-way spectral clustering**](https://doi.org/10.1093/imaiai/iay008)[”](https://drive.google.com/file/d/1eWIeV12C78cthlB7D498g3pq_QvtCofI/view?usp=share_link), 2019

#### 2.3.5.2. Graphiques de regroupement spectral

Le clustering spectral peut également être utilisé pour partitionner des graphiques via leurs intégrations spectrales. Dans ce cas, la matrice d'affinité est la matrice d'adjacence du graphe, et SpectralClustering est initialisé avec `affinity='precomputed'` :

In [ ]:
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(3, affinity='precomputed', n_init=100,
                        assign_labels='discretize')
sc.fit_predict(adjacency_matrix)

##### Références



[?] Ulrike von Luxburg,  [“**A Tutorial on Spectral Clustering**](https://arxiv.org/pdf/0711.0189.pdf)[”](https://drive.google.com/file/d/1Hb7HwNENVUXgosAk3bG0tpcM-AhfjQdF/view?usp=share_link), 2007

[?] Jianbo Shi, Jitendra Malik, [“**Normalized cuts and image segmentation**](https://people.eecs.berkeley.edu/~malik/papers/SM-ncut.pdf)[”](https://drive.google.com/file/d/1-_p-J8SktpTVCdTs140WRNpJ5qPFHbzD/view?usp=share_link), 2000

[?] Marina Meila, Jianbo Shi, [“**A Random Walks View of Spectral Segmentation**](https://www.ri.cmu.edu/pub_files/pub3/maila_marina_2001_2/maila_marina_2001_2.pdf)[”](https://drive.google.com/file/d/19zsTu3Vz1NtibSD2qw_3u44uZYvsj3na/view?usp=share_link), 2001

[?] Andrew Y. Ng, Michael I. Jordan, Yair Weiss, [“**On Spectral Clustering: Analysis and an algorithm**](https://proceedings.neurips.cc/paper/2001/file/801272ee79cfde7fa5960571fee36b9b-Paper.pdf)[”](https://drive.google.com/file/d/1DNbfVClFKFNdZ7ubynCUbGSvPdtUHRJG/view?usp=share_link), 2001

[?] David Zhuzhunashvili, Andrew Knyazev, [“**Preconditioned Spectral Clustering for Stochastic Block Partition Streaming Graph Challenge**](https://arxiv.org/pdf/1708.07481.pdf)[”](https://drive.google.com/file/d/1QhXpbu371LnDwzTeIpidRr-YCWRA-ZJE/view?usp=share_link)



## 2.3.6. Classification hiérarchique

Le clustering hiérarchique est une famille générale d'algorithmes de clustering qui construisent des clusters imbriqués en les fusionnant ou en les divisant successivement. Cette hiérarchie de clusters est représentée sous forme d'arbre (ou dendrogramme). La racine de l'arbre est la grappe unique qui rassemble tous les échantillons, les feuilles étant les grappes à un seul échantillon. Voir la [**page Wikipédia**](https://en.wikipedia.org/wiki/Hierarchical_clustering) pour plus de détails.

L'objet [**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) effectue un clustering hiérarchique en utilisant une approche ascendante : chaque observation commence dans son propre cluster, et les clusters sont successivement fusionnés. Les critères de liaison déterminent la métrique utilisée pour la stratégie de fusion :

* **Ward** minimise la somme des différences au carré dans tous les clusters. Il s'agit d'une approche de minimisation de la variance et, en ce sens, elle est similaire à la fonction objectif k-means mais abordée avec une approche hiérarchique agglomérative.
* **La liaison maximale** ou **complète** minimise la distance maximale entre les observations de paires de grappes.
* **Le couplage moyen** minimise la moyenne des distances entre toutes les observations de paires de grappes.
* **Le couplage unique** minimise la distance entre les observations les plus proches des paires de grappes.

L'[**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) peut également évoluer vers un grand nombre d'échantillons lorsqu'il est utilisé conjointement avec une matrice de connectivité, mais est coûteux en calcul lorsqu'aucune contrainte de connectivité n'est ajoutée entre les échantillons : il considère à chaque étape toutes les fusions possibles.

[**`FeatureAgglomeration`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

Le [**`FeatureAgglomeration`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) utilise le clustering agglomératif pour regrouper les caractéristiques qui semblent très similaires, réduisant ainsi le nombre de caractéristiques. C'est un outil de réduction de dimensionnalité, voir [**Réduction de dimensionnalité non supervisée** (6.5)](https://scikit-learn.org/stable/modules/unsupervised_reduction.html#data-reduction).

### 2.3.6.1. Différents types de liaison : Ward, complète, moyenne et simple liaison

L'[**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) prend en charge les stratégies de liaison Ward, unique, moyenne et complète.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_linkage_comparison_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_linkage_comparison_001.png" style="width: 589.1px; height: 623.5px;" />

Le cluster agglomératif a un comportement « riche devient plus riche » qui conduit à des tailles de cluster inégales. À cet égard, la liaison simple est la pire stratégie, et Ward donne les tailles les plus régulières. Cependant, l'affinité (ou la distance utilisée dans le clustering) ne peut pas être modifiée avec Ward, donc pour les métriques non euclidiennes, la liaison moyenne est une bonne alternative. La liaison unique, bien qu'elle ne soit pas robuste aux données bruitées, peut être calculée très efficacement et peut donc être utile pour fournir un regroupement hiérarchique d'ensembles de données plus volumineux. La liaison simple peut également bien fonctionner sur des données non globulaires.

#### Exemples

##### [**Divers clustering agglomératifs sur un plongement 2D de chiffres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_digits_linkage.ipynb)<br/>([*Various Agglomerative Clustering on a 2D embedding of digits*](https://scikit-learn.org/stable/auto_examples/cluster/plot_digits_linkage.html))

Exploration des différentes stratégies de liaison dans un jeu de données réel.

### 2.3.6.2. Visualisation de la hiérarchie des clusters

Il est possible de visualiser l'arbre représentant la fusion hiérarchique des clusters sous forme de dendrogramme. L'inspection visuelle peut souvent être utile pour comprendre la structure des données, mais plus encore dans le cas d'échantillons de petite taille.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_dendrogram_001.png)

### 2.3.6.3. Ajout de contraintes de connectivité

Un aspect intéressant d'[**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) est que des contraintes de connectivité peuvent être ajoutées à cet algorithme (seuls les clusters adjacents peuvent être fusionnés), via une matrice de connectivité qui définit pour chaque échantillon les échantillons voisins suivant une structure particulière des données. Par exemple, dans l'exemple du rouleau suisse ci-dessous, les contraintes de connectivité interdisent la fusion de points qui ne sont pas adjacents sur le rouleau suisse, et évitent ainsi la formation de clusters qui s'étendent sur des plis superposés du rouleau.

<img alt="unstructured" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_ward_structured_vs_unstructured_001.png" style="width: 313.6px; height: 235.2px;" /><img alt="structured" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_ward_structured_vs_unstructured_002.png" style="width: 313.6px; height: 235.2px;" />

Ces contraintes sont utiles pour imposer une certaine structure locale, mais elles rendent aussi l'algorithme plus rapide, surtout lorsque le nombre d'échantillons est élevé.

Les contraintes de connectivité sont imposées via une matrice de connectivité : une matrice creuse scipy qui n'a d'éléments qu'à l'intersection d'une ligne et d'une colonne avec des indices de l'ensemble de données qui doivent être connectés. Cette matrice peut être construite à partir d'informations a priori : par exemple, vous pouvez regrouper des pages Web en ne fusionnant que des pages avec un lien pointant de l'une vers l'autre. Il peut également être appris à partir des données, par exemple en utilisant [**`sklearn.neighbors.kneighbors_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html) pour limiter la fusion aux voisins les plus proches comme dans [**cet exemple**](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering.html), ou en utilisant [**`sklearn.feature_extraction.image.grid_to_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.grid_to_graph.html) pour activer uniquement la fusion des pixels voisins sur une image, comme dans l'[**exemple de la pièce**](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_ward_segmentation.html).

#### Exemples

#### [**Une démo de clustering hiérarchique Ward structuré sur une image de pièces**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_coin_ward_segmentation.ipynb)<br/>([*A demo of structured Ward hierarchical clustering on an image of coins*](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_ward_segmentation.html))

Ward clustering pour diviser l'image des pièces en régions.

#### [**Clustering hiérarchique : ward structuré vs non structuré**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_coin_ward_segmentation.ipynb)<br/>([*Hierarchical clustering: structured vs unstructured ward*](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_ward_segmentation.html))

Exemple d'algorithme de Ward sur un rouleau suisse, comparaison des approches structurées versus des approches non structurées.

#### [**Agglomération de caractéristiques vs sélection univariée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_feature_agglomeration_vs_univariate_selection.ipynb)<br/>([*Feature agglomeration vs. univariate selection*](https://scikit-learn.org/stable/auto_examples/cluster/plot_feature_agglomeration_vs_univariate_selection.html))

Exemple de réduction de dimensionnalité avec agglomération de caractéristiques basée sur le regroupement hiérarchique de Ward.

#### [**Agglomération avec et sans structure**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_agglomerative_clustering.ipynb)<br/>([*Agglomerative clustering with and without structure*](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering.html))

#### Avertissement

**Contraintes de connectivité avec liaison simple, moyenne et complète**

Les contraintes de connectivité et les liaisons simples, complètes ou moyennes peuvent améliorer l'aspect 'de plus en plus riche' du clustering agglomératif, en particulier s'ils sont construits avec [**`sklearn.neighbors.kneighbors_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html). Dans la limite d'un petit nombre d'amas, ils tendent à donner quelques amas macroscopiquement occupés et presque vides. (voir la discussion dans [**Agglomération avec et sans structure**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_agglomerative_clustering.ipynb)). La liaison unique est l'option de liaison la plus fragile en ce qui concerne cette question.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_001.png" style="width: 380.0px; height: 152.0px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_002.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_002.png" style="width: 380.0px; height: 152.0px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_003.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_003.png" style="width: 380.0px; height: 152.0px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_004.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_004.png" style="width: 380.0px; height: 152.0px;" />


### 2.3.6.4. Varier la métrique

Une liaison simple, moyenne et complète peut être utilisée avec une variété de distances (ou affinités), en particulier la distance euclidienne ($\ell_2$), la distance de Manhattan (ou Cityblock, ou $\ell_1$), la distance cosinus ou toute matrice d'affinité précalculée.
* La distance $\ell_1$ est souvent bonne pour les caractéristiques creux ou le bruit creux: c'est-à-dire que de nombreuses caractéristiques sont nulles, comme dans l'exploration de texte utilisant des occurrences de mots rares.
* La distance cosinus est intéressante car elle est invariante aux échelles globales du signal.

Les directives pour choisir une métrique sont d'en utiliser une qui maximise la distance entre les échantillons dans différentes classes et la minimise dans chaque classe.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_005.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_005.png" style="width: 204.8px; height: 153.6px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_006.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_006.png" style="width: 204.8px; height: 153.6px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_007.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_007.png" style="width: 204.8px; height: 153.6px;" />

#### Exemples

##### [**Clustering agglomératif avec différentes métriques**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_agglomerative_clustering_metrics.ipynb)<br/>([*Agglomerative clustering with different metrics*](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering_metrics.html))


### 2.3.6.5. Bisecting K-Means

Le [**`BisectingKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.BisectingKMeans.html) est une variante itérative de [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), utilisant le clustering hiérarchique de division. Au lieu de créer tous les centroïdes en même temps, les centroïdes sont sélectionnés progressivement en fonction d'un clustering précédent : un cluster est divisé en deux nouveaux clusters à plusieurs reprises jusqu'à ce que le nombre cible de clusters soit atteint.

[**`BisectingKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.BisectingKMeans.html) est plus efficace que [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) lorsque le nombre de clusters est grand car il ne fonctionne que sur un sous-ensemble de données à chaque bissection tandis que [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) fonctionne toujours sur l'ensemble de données entier.

Bien que [**`BisectingKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.BisectingKMeans.html) ne puisse pas bénéficier des avantages de l'initialisation `"k-means++"` par conception, il produira toujours des résultats comparables à `KMeans(init="k-means++")` en termes d'inertie à des coûts de calcul moins chers, et produira probablement de meilleurs résultats que `KMeans` avec une initialisation aléatoire.

Cette variante est plus efficace pour le clustering agglomératif si le nombre de clusters est petit par rapport au nombre de points de données.

Cette variante ne produit pas non plus de clusters vides.

**Il existe deux stratégies pour sélectionner le cluster à scinder :**
* `bisecting_strategy="largest_cluster"` sélectionne le cluster ayant le plus de points
* `bisecting_strategy="biggest_inertia"` sélectionne le cluster avec la plus grande inertie (cluster avec la plus grande somme des erreurs au carré à l'intérieur)

La sélection par la plus grande quantité de points de données dans la plupart des cas produit un résultat aussi précis que la sélection par inertie et est plus rapide (en particulier pour une plus grande quantité de points de données, où l'erreur de calcul peut être coûteuse).

La sélection par la plus grande quantité de points de données produira également probablement des clusters de tailles similaires, tandis que `KMeans` est connu pour produire des clusters de tailles différentes.

La différence entre Bisecting K-Means et Regular K-Means peut être vue sur l'exemple [**Bisecting K-Means et Regular K-Means Comparaison des performances**](https://scikit-learn.org/stable/auto_examples/cluster/plot_bisect_kmeans.html). Alors que l'algorithme K-Means régulier a tendance à créer des clusters non liés, les clusters de Bisecting K-Means sont bien ordonnés et créent une hiérarchie assez visible.

#### Références

[?] Michael Steinbach, George Karypis and Vipin Kumar, [“**A Comparison of Document Clustering Techniques**](http://www.philippe-fournier-viger.com/spmf/bisectingkmeans.pdf)[”](https://drive.google.com/file/d/1XwuyijuSlSEzdhS3UUzxfWB-K16YLoMs/view?usp=share_link), Department of Computer Science and Egineering, University of Minnesota (June 2000)

[?] K.Abirami and Dr.P.Mayilvahanan, [“**Performance Analysis of K-Means and Bisecting K-Means Algorithms in Weblog Data**](https://ijeter.everscience.org/Manuscripts/Volume-4/Issue-8/Vol-4-issue-8-M-23.pdf)[”](https://drive.google.com/file/d/14ypP25t0s7Oxqg15aDqjF0T5fatoz1SM/view?usp=share_link), International Journal of Emerging Technologies in Engineering Research (IJETER) Volume 4, Issue 8, (August 2016)

[?] Jian Di, Xinyue Gou, [“**Bisecting K-means Algorithm Based on K-valued Self-determining and Clustering Center Optimization**](http://www.jcomputers.us/vol13/jcp1306-01.pdf)[”](https://drive.google.com/file/d/1pTCDwE70kjr1y7GmIYfJNatHGhU58tQe/view?usp=share_link), School of Control and Computer Engineering, North China Electric Power University, Baoding, Hebei, China (August 2017)


## 2.3.7. DBSCAN

L'algorithme [**`DBSCAN`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) considère les clusters comme des zones de haute densité séparées par des zones de faible densité. En raison de cette vue plutôt générique, les clusters trouvés par DBSCAN peuvent avoir n'importe quelle forme, contrairement à k-means qui suppose que les clusters sont de forme convexe. L'élément central du DBSCAN est le concept d'**échantillons centraux** (*core samples*), qui sont des échantillons qui se trouvent dans des zones à haute densité. Une grappe est donc un ensemble d'échantillons centraux, chacun proche l'un de l'autre (mesuré par une mesure de distance) et un ensemble d'échantillons non centrux qui sont proches d'un échantillon central (mais ne sont pas eux-mêmes des échantillons centraux). Il y a deux paramètres dans l'algorithme, `min_samples` et `eps`, qui définissent formellement ce que nous voulons dire quand nous disons *dense*. Des `min_samples` plus élevés ou des `eps` plus faibles indiquent une densité plus élevée nécessaire pour former un cluster.

Plus formellement, nous définissons un échantillon central comme étant un échantillon dans l'ensemble de données tel qu'il existe `min_samples` d'autres échantillons à une distance de `eps`, qui sont définis comme voisins de l'échantillon central. Cela nous indique que l'échantillon central se trouve dans une zone dense de l'espace vectoriel. Un cluster est un ensemble d'échantillons centraux qui peuvent être construits en prenant de manière récursive un échantillon central, en trouvant tous ses voisins qui sont des échantillons centraux, en trouvant tous leurs voisins qui sont des échantillons centraux, etc. Un cluster a également un ensemble d'échantillons non centraux, qui sont des échantillons voisins d'un échantillon central dans le cluster mais qui ne sont pas eux-mêmes des échantillons centraux. Intuitivement, ces échantillons sont en marge d'un cluster.

Tout échantillon central fait partie d'une grappe, par définition. Tout échantillon qui n'est pas un échantillon central et qui se trouve à une distance d'au moins `eps` de tout échantillon central est considéré comme une valeur aberrante par l'algorithme.

Alors que le paramètre `min_samples` contrôle principalement la tolérance de l'algorithme au bruit (sur des ensembles de données bruyants et volumineux, il peut être souhaitable d'augmenter ce paramètre), le paramètre `eps` est *crucial pour choisir de manière appropriée*  l'ensemble de données et la fonction de distance et ne peut généralement pas être laissé à sa valeur par défaut. Il contrôle le voisinage local des points. Lorsque cette valeur est trop petite, la plupart des données ne seront pas du tout regroupées (et étiquetées à `-1` pour « bruit »). Lorsqu'elle est trop grande, celà provoque la fusion des clusters proches en un seul cluster, et finalement le retour de l'ensemble de données complet en un seul cluster. Certaines heuristiques pour choisir ce paramètre ont été discutées dans la littérature, par exemple basées sur un genou dans le tracé des distances du plus proche voisin (comme discuté dans les références ci-dessous).

Dans la figure ci-dessous, la couleur indique l'appartenance au cluster, avec de grands cercles indiquant les échantillons centraux trouvés par l'algorithme. Les cercles plus petits sont des échantillons non centraux qui font toujours partie d'un cluster. De plus, les valeurs aberrantes sont indiquées par des points noirs ci-dessous.

<img alt="dbscan_results" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_dbscan_001.png" style="width: 320.0px; height: 240.0px;" />

### Exemple

#### [**Démo de l'algorithme de clustering DBSCAN**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_dbscan.ipynb)<br/>([*Demo of DBSCAN clustering algorithm*](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html))

### Implémentation

L'algorithme DBSCAN est déterministe, générant toujours les mêmes clusters lorsqu'on lui donne les mêmes données dans le même ordre. Cependant, les résultats peuvent différer lorsque les données sont fournies dans un ordre différent. Premièrement, même si les échantillons centraux seront toujours affectés aux mêmes grappes, les étiquettes de ces grappes dépendent de l'ordre dans lequel ces échantillons sont rencontrés dans les données. Deuxièmement et plus important encore, les grappes auxquelles les échantillons non centraux sont affectés peuvent différer selon l'ordre des données. Cela se produit lorsqu'un échantillon non central a une distance inférieure à `eps` à deux échantillons centraux dans des grappes différentes. Par l'inégalité triangulaire, ces deux échantillons centraux doivent être plus éloignés que `eps` l'un de l'autre, sinon ils seraient dans le même groupe. L'échantillon non central est affecté à la grappe générée en premier lors d'un passage à travers les données, et les résultats dépendent donc de l'ordre des données.

L'implémentation actuelle utilise des arbres à billes et des kd-arbres  pour déterminer le voisinage des points, ce qui évite de calculer la matrice de distance complète (comme cela était fait dans les versions scikit-learn antérieures à 0.14). La possibilité d'utiliser des métriques personnalisées est conservée ; pour plus de détails, voir `NearestNeighbors`.

### Consommation mémoire pour les échantillons de grande taille

Cette implémentation n'est par défaut pas efficace en mémoire car elle construit une matrice de similarité complète par paires dans le cas où les kd-arbres ou les arbres à billes ne peuvent pas être utilisés (par exemple, avec des matrices creuses). Cette matrice consommera $n^2$ nombres flottants. Voici quelques mécanismes pour contourner ce problème :
* Utiliser le clustering [**OPTICS** (2.3.8)](https://scikit-learn.org/stable/modules/clustering.html#optics) conjointement avec la méthode `extract_dbscan`. Le clustering OPTICS calcule également la matrice complète par paires, mais ne conserve qu'une seule ligne en mémoire à la fois (complexité de la mémoire n).
* Un graphe de voisinage à rayon creux (où les entrées manquantes sont supposées être hors `eps`) peut être précalculé d'une manière économe en mémoire et dbscan peut être exécuté dessus avec `metric='precomputed'`. Voir [**`sklearn.neighbors.NearestNeighbors.radius_neighbors_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html).
* L'ensemble de données peut être compressé, soit en supprimant les doublons exacts s'ils se produisent dans vos données, soit en utilisant BIRCH. Ensuite, vous n'avez qu'un nombre relativement restreint de représentants pour un grand nombre de points. Vous pouvez ensuite fournir un `sample_weight` lors de l'ajustement de DBSCAN.

### Références

[?] Ester, M., H. P. Kriegel, J. Sander, and X. Xu, [“**A Density-Based Algorithm for Discovering Clusters in Large Spatial Databases with Noise**](https://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf)[”](https://drive.google.com/file/d/13RAM29dEJVWiyUjg1rBilvrT7k02YCl_/view?usp=share_link), In Proceedings of the 2nd International Conference on Knowledge Discovery and Data Mining, Portland, OR, AAAI Press, pp. 226–231. 1996

[?] Schubert, E., Sander, J., Ester, M., Kriegel, H. P., & Xu, X., [“**DBSCAN revisited, revisited: why and how you should (still) use DBSCAN**](https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/notes_slides/revisitofrevisitDBSCAN.pdf)[”](https://drive.google.com/file/d/1d1W-6NPSbIiQR0cEGxzsEHkqvjTRixFZ/view?usp=share_link), In ACM Transactions on Database Systems (TODS), 42(3), 19, 2017


## 2.3.8. OPTICS

L'algorithme [**`OPTICS`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html) partage de nombreuses similitudes avec l'algorithme [**`DBSCAN`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) et peut être considéré comme une généralisation de DBSCAN qui assouplit l'exigence `eps` d'une valeur unique à une plage de valeurs. La principale différence entre DBSCAN et OPTICS est que l'algorithme OPTICS construit un graphique d'*accessibilité*, qui attribue à chaque échantillon à la fois une distance de `reachability_` et un emplacement dans l'attribut `ordering_` de cluster ; ces deux attributs sont affectés lors de l'ajustement du modèle et sont utilisés pour déterminer l'appartenance au cluster. Si OPTICS est exécuté avec `max_eps` défini à la valeur par défaut *inf*, l'extraction de cluster de style DBSCAN peut être effectuée de manière répétée en temps linéaire pour toute valeur `eps` donnée à l'aide de la méthode `cluster_optics_dbscan`. Définir `max_eps` sur une valeur inférieure entraînera des temps d'exécution plus courts et peut être considéré comme le rayon de voisinage maximal à partir de chaque point pour trouver d'autres points potentiellement accessibles.

<img alt="optics_results" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_optics_001.png" style="width: 500.0px; height: 350.0px;" />

Les distances d'*accessibilité* générées par OPTICS permettent une extraction à densité variable de clusters dans un seul ensemble de données. Comme le montre le graphique ci-dessus, la combinaison des distances d'*accessibilité* et d'`ordering_` des ensembles de données produit un *graphique d'accessibilité*, où la densité de points est représentée sur l'axe Y, et où les points sont ordonnés de telle sorte que les points proches soient adjacents. "Couper" le graphique d'accessibilité à une seule valeur produit des résultats de type DBSCAN ; tous les points au-dessus de la «coupe» sont classés comme du bruit, et chaque fois qu'il y a une pause lors de la lecture de gauche à droite signifie un nouveau cluster. L'extraction de cluster par défaut avec OPTICS examine les pentes raides dans le graphique pour trouver des clusters, et l'utilisateur peut définir ce qui compte comme une pente raide à l'aide du paramètre `xi`. Il existe également d'autres possibilités d'analyse sur le graphique lui-même, telles que la génération de représentations hiérarchiques des données via des dendrogrammes d'accessibilité, et la hiérarchie des clusters détectés par l'algorithme est accessible via le paramètre `cluster_hierarchy_`. Le tracé ci-dessus a été codé par couleur afin que les couleurs des clusters dans l'espace planaire correspondent aux clusters de segments linéaires du tracé d'accessibilité. Notez que les clusters bleu et rouge sont adjacents dans le tracé d'accessibilité et peuvent être représentés hiérarchiquement comme des enfants d'un cluster parent plus grand.

### Exemples

#### [**Démo de l'algorithme de clustering OPTICS**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_optics.ipynb)<br/>([*Demo of OPTICS clustering algorithm*](https://scikit-learn.org/stable/auto_examples/cluster/plot_optics.html))

### Comparaison avec DBSCAN

Les résultats de la méthode OPTICS `cluster_optics_dbscan` et de DBSCAN sont très similaires, mais pas toujours identiques ; plus précisément, l'étiquetage de la périphérie et des points de bruit. Cela s'explique en partie par le fait que les premiers échantillons de chaque zone dense traitée par OPTICS ont une grande valeur d'accessibilité tout en étant proches d'autres points de leur zone, et seront donc parfois marqués comme bruit plutôt que comme périphérie. Cela affecte les points adjacents lorsqu'ils sont considérés comme des candidats pour être marqués comme périphérie ou bruit.

Notez que pour toute valeur unique d'`eps`, DBSCAN aura tendance à avoir un temps d'exécution plus court que OPTICS ; cependant, pour des exécutions répétées à différentes valeurs `eps`, une seule exécution d'OPTICS peut nécessiter moins de temps d'exécution cumulé que DBSCAN. Il est également important de noter que la sortie d'OPTICS est proche de celle de DBSCAN uniquement si `eps` et `max_eps` sont proches.

### Complexité algorithmique

Les arbres d'indexation spatiale sont utilisés pour éviter de calculer la matrice de distance complète et permettent une utilisation efficace de la mémoire sur de grands ensembles d'échantillons. Différentes métriques de distance peuvent être fournies via le mot-clé `metric`.

Pour les grands ensembles de données, des résultats similaires (mais pas identiques) peuvent être obtenus via [**`HDBSCAN`**](https://hdbscan.readthedocs.io/en/latest/). L'implémentation HDBSCAN est multithreadée et a une meilleure complexité algorithmique qu'OPTICS, au prix d'une plus mauvaise mise à l'échelle de la mémoire. Pour les ensembles de données extrêmement volumineux qui épuisent la mémoire du système à l'aide de HDBSCAN, OPTICS conservera $n$
(par opposition à $n^2$) mise à l'échelle de la mémoire ; cependant, le réglage du paramètre `max_eps` devra probablement être utilisé pour donner une solution dans un délai raisonnable.

### Références:

[?] Ankerst, Mihael, Markus M. Breunig, Hans-Peter Kriegel et Jörg Sander., [“**OPTICS: ordering points to identify the clustering structure**](https://www.dbs.ifi.lmu.de/Publikationen/Papers/OPTICS.pdf)[”](https://drive.google.com/file/d/1U46lCsWPejLFLlWRBqqpYJnUsqfbNarg/view?usp=share_link), In ACM Sigmod Record, vol. 28, non. 2, p. 49-60. AMC, 1999.

## 2.3.9. BIRCH

Le [**`Birch`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) construit un arbre appelé Clustering Feature Tree (CFT) pour les données données. Les données sont essentiellement compressées avec perte vers un ensemble de nœuds de caractéristiques de clustering (nœuds CF). Les nœuds CF ont un certain nombre de sous-clusters appelés sous-clusters de caractéristiques de clustering (sous-clusters CF) et ces sous-clusters CF situés dans les nœuds CF non terminaux peuvent avoir des nœuds CF comme enfants.

Les sous-grappes CF contiennent les informations nécessaires au regroupement, ce qui évite d'avoir à conserver toutes les données d'entrée en mémoire. Ces informations comprennent :
* Nombre d'échantillons dans un sous-cluster.
* Somme linéaire - Un vecteur à n dimensions contenant la somme de tous les échantillons
* Somme au carré - Somme de la norme L2 au carré de tous les échantillons.
* Centroïdes - Pour éviter le recalcul somme linéaire / n_samples.
* Norme au carré des centroïdes.

L'algorithme BIRCH a deux paramètres, le seuil et le facteur de branchement. Le facteur de branchement limite le nombre de sous-grappes dans un nœud et le seuil limite la distance entre l'échantillon entrant et les sous-grappes existantes.

Cet algorithme peut être considéré comme une instance ou une méthode de réduction de données, car il réduit les données d'entrée à un ensemble de sous-clusters qui sont obtenus directement à partir des feuilles du CFT. Ces données réduites peuvent être traitées ultérieurement en les introduisant dans un cluster global. Ce clusterer global peut être défini par `n_clusters`. Si `n_clusters` est défini sur `None`, les sous-clusters des feuilles sont directement lus, sinon une étape de clustering global étiquette ces sous-clusters en clusters globaux (étiquettes) et les échantillons sont mappés sur l'étiquette globale du sous-cluster le plus proche.

### Description de l'algorithme :

* Un nouvel échantillon est inséré dans la racine de l'arborescence CF qui est un nœud CF. Il est ensuite fusionné avec le sous-cluster de la racine, qui a le plus petit rayon après fusion, contraint par les conditions de seuil et de facteur de branchement. Si le sous-cluster a un nœud enfant, cela est répété jusqu'à ce qu'il atteigne une feuille. Après avoir trouvé le sous-cluster le plus proche dans la feuille, les propriétés de ce sous-cluster et des sous-clusters parents sont mises à jour de manière récursive.
* Si le rayon du sous-groupe obtenu en fusionnant le nouvel échantillon et le sous-groupe le plus proche est supérieur au carré du seuil et si le nombre de sous-groupes est supérieur au facteur de branchement, alors un espace est temporairement alloué à ce nouvel échantillon. Les deux sous-clusters les plus éloignés sont pris et les sous-clusters sont divisés en deux groupes sur la base de la distance entre ces sous-clusters.
* Si ce nœud divisé a un sous-cluster parent et qu'il y a de la place pour un nouveau sous-cluster, alors le parent est divisé en deux. S'il n'y a pas de place, ce nœud est à nouveau divisé en deux et le processus se poursuit de manière récursive, jusqu'à ce qu'il atteigne la racine.

### BIRCH ou MiniBatchKMeans ?

* BIRCH ne s'adapte pas très bien aux données de grande dimension. En règle générale, si `n_features` est supérieur à vingt, il est généralement préférable d'utiliser MiniBatchKMeans.
* Si le nombre d'instances de données doit être réduit, ou si l'on souhaite un grand nombre de sous-clusters, soit comme étape de prétraitement, soit autrement, BIRCH est plus utile que MiniBatchKMeans.

### Comment utiliser partial_fit ?

Pour éviter le calcul du clustering global, pour chaque appel de `partial_fit` il est conseillé de :
1. Définir initialement `n_clusters=None`
2. Entraîner toutes les données par plusieurs appels à `partial_fit`.
3. Définir `n_clusters` sur une valeur requise à l'aide de `brc.set_params(n_clusters=n_clusters)`.
4. Appeler enfin `partial_fit` sans arguments, c'est-à-dire `brc.partial_fit()` qui effectue le clustering global.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_birch_vs_minibatchkmeans_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_birch_vs_minibatchkmeans_001.png" />


### Exemples

#### [**Compare BIRCH and MiniBatchKMeans**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_birch_vs_minibatchkmeans.ipynb)<br/>([*Comparez BIRCH et MiniBatchKMeans*](https://scikit-learn.org/stable/auto_examples/cluster/plot_birch_vs_minibatchkmeans.html))

### Références

[?] Tian Zhang, Raghu Ramakrishnan, Maron Livny [“**BIRCH: An efficient data clustering method for large databases**](https://www.cs.sfu.ca/CourseCentral/459/han/papers/zhang96.pdf)[”](https://drive.google.com/file/d/1n8-C65wdcXpHHrErzrAswXeuuBc24uas/view?usp=share_link)

[?] Roberto Perdisci JBirch - [Java implementation of BIRCH clustering algorithm](https://code.google.com/archive/p/jbirch)


# 2.3.10. Évaluation des performances de clustering

Évaluer les performances d'un algorithme de clustering n'est pas aussi simple que compter le nombre d'erreurs ou la précision et le rappel d'un algorithme de classification supervisée. En particulier, plutôt que de prendre en compte les valeurs absolues des étiquettes de cluster, une métrique d'évaluation meure si ce regroupement définit des séparations des données similaires à un ensemble de classes de vérité terrain ou satisfaisant à une hypothèse telle que les membres appartenant à la même classe sont plus similaires que les membres de classes différentes selon une métrique de similarité.

## 2.3.10.1. Indice Rand

Compte tenu de la connaissance des affectations de classe de vérité terrain `labels_true` et de nos affectations d'algorithme de regroupement des mêmes échantillons `labels_pred`, l'**indice Rand (ajusté ou non ajusté)** est une fonction qui mesure la **similarité** des deux affectations, en ignorant les permutations :

In [5]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]
metrics.rand_score(labels_true, labels_pred)
# 0.66...

0.6666666666666666

L'indice Rand ne garantit pas d'obtenir une valeur proche de 0,0 pour un étiquetage aléatoire. L'indice Rand ajusté **corrige le hasard** et donnera une telle valeur de référence.

In [6]:
metrics.adjusted_rand_score(labels_true, labels_pred)
# 0.24...

0.24242424242424243

Comme pour toutes les métriques de clustering, on peut permuter 0 et 1 dans les libellés prédits, renommer 2 en 3 et obtenir le même score :

In [7]:
labels_pred = [1, 1, 0, 0, 3, 3]
metrics.rand_score(labels_true, labels_pred)
# 0.66...
metrics.adjusted_rand_score(labels_true, labels_pred)
# 0.24...

0.24242424242424243

De plus, [**`rand_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.rand_score.html) et [**`ajusted_rand_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html) sont tous deux **symétriques** : l'échange d'arguments ne modifie pas les scores. Ils peuvent ainsi être utilisés comme **mesures de consensus** :

In [8]:
metrics.rand_score(labels_pred, labels_true)
# 0.66...
metrics.adjusted_rand_score(labels_pred, labels_true)
# 0.24...

0.24242424242424243

L'étiquetage parfait est noté 1,0 :

In [9]:
labels_pred = labels_true[:]
metrics.rand_score(labels_true, labels_pred)
# 1.0
metrics.adjusted_rand_score(labels_true, labels_pred)
# 1.0

1.0

Les étiquettes peu concordantes (par exemple, les étiquettes indépendantes) ont des scores inférieurs, et pour l'indice Rand ajusté, le score sera négatif ou proche de zéro. Cependant, pour l'indice Rand non ajusté, le score, bien qu'inférieur, ne sera pas nécessairement proche de zéro :

In [10]:
labels_true = [0, 0, 0, 0, 0, 0, 1, 1]
labels_pred = [0, 1, 2, 3, 4, 5, 5, 6]
metrics.rand_score(labels_true, labels_pred)
# 0.39...
metrics.adjusted_rand_score(labels_true, labels_pred)
# -0.07...

-0.07207207207207207

### Avantages

* **Interprétabilité** : l'indice Rand non ajusté est proportionnel au nombre de paires d'échantillons dont les étiquettes sont identiques dans `labels_pred` et `labels_true`, ou sont différentes dans les deux.
* **Les attributions d'étiquettes aléatoires (uniformes)** ont un score d'indice Rand ajusté proche de 0,0 pour toute valeur de `n_clusters` et `n_samples` (ce qui n'est pas le cas pour l'indice Rand non ajusté ou la V-mesure par exemple).
* **Plage délimitée** : les valeurs inférieures indiquent des étiquetages différents, des regroupements similaires ont un indice Rand élevé (ajusté ou non ajusté), 1,0 est le score de correspondance parfaite. La plage de score est [0, 1] pour l'indice Rand non ajusté et [-1, 1] pour l'indice Rand ajusté.
* **Aucune hypothèse n'est faite sur la structure du cluster** : l'indice Rand (ajusté ou non ajusté) peut être utilisé pour comparer toutes sortes d'algorithmes de clustering, et peut être utilisé pour comparer des algorithmes de clustering tels que k-means qui suppose des formes de blob isotropes avec des résultats d'analyse spectrale. algorithmes de clustering qui peuvent trouver des clusters avec des formes "pliées".

### Inconvénients

* Contrairement à l'inertie, l'**indice Rand (ajusté ou non) nécessite une connaissance des classes de vérité terrain** qui n'est presque jamais disponible dans la pratique ou nécessite une affectation manuelle par des annotateurs humains (comme dans le cadre de l'apprentissage supervisé).
Cependant, l'indice Rand (ajusté ou non ajusté) peut également être utile dans un cadre purement non supervisé en tant que bloc de construction d'un indice de consensus pouvant être utilisé pour la sélection de modèles de clustering (TODO).

* L'**indice Rand non ajusté est souvent proche de 1,0** même si les regroupements eux-mêmes diffèrent considérablement. Cela peut être compris lors de l'interprétation de l'indice Rand comme la précision de l'étiquetage des paires d'éléments résultant des regroupements : dans la pratique, il y a souvent une majorité de paires d'éléments qui se voient attribuer l'étiquette de paire différente sous le regroupement prédit et la vérité terrain, ce qui entraîne un forte proportion d'étiquettes de paires concordantes, ce qui conduit par la suite à un score élevé.

### Exemples

#### [**Ajustement pour le hasard dans l'évaluation des performances de clustering**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_adjusted_for_chance_measures.ipynb)<br/>([*Adjustment for chance in clustering performance evaluation*](https://scikit-learn.org/stable/auto_examples/cluster/plot_adjusted_for_chance_measures.html))


### Formulation mathématique

Si C est une affectation de classe de vérité terrain et K le regroupement, définissons $a$ et $b$ comme:
* $a$, le nombre de paires d'éléments qui sont dans le même ensemble en C et dans le même ensemble en K
* $b$, le nombre de paires d'éléments qui sont dans des ensembles différents dans C et dans des ensembles différents dans K

L'indice Rand non ajusté est alors donné par :

$$\text{RI} = \frac{a + b}{C_2^{n_{samples}}}$$

où $C_2^{n_{samples}}$ est le nombre total de paires possibles dans le jeu de données. Peu importe si le calcul est effectué sur des paires ordonnées ou des paires non ordonnées tant que le calcul est effectué de manière cohérente.

Cependant, l'indice Rand ne garantit pas que les attributions aléatoires d'étiquettes obtiendront une valeur proche de zéro (surtout si le nombre de grappes est du même ordre de grandeur que le nombre d'échantillons).

Pour contrer cet effet, nous pouvons actualiser le RI attendu $E[\text{RI}]$ d'étiquetages aléatoires en définissant l'indice Rand ajusté comme suit :

$$\text{ARI} = \frac{\text{RI} - E[\text{RI}]}{\max(\text{RI}) - E[\text{RI}]}$$

### Références

[?] L. Hubert and P. Arabie, “**Comparing Partitions**”, Journal of Classification, 1985[.](https://drive.google.com/file/d/1tLtEWnFFD6PZ_uv8uwI_UNTBvZQTa0og/view?usp=share_link)

[?] D. Steinley, “**Properties of the Hubert-Arabie adjusted Rand index**”, Psychological Methods, 2004[.](https://drive.google.com/file/d/14PJfGnsbax-DtX_guyQw6UDeBbn-O458/view?usp=share_link)

[?] [Wikipedia entry for the **Rand index**](https://en.wikipedia.org/wiki/Rand_index)

[?] [Wikipedia entry for the **adjusted Rand index**](https://en.wikipedia.org/wiki/Rand_index#Adjusted_Rand_index)


## 2.3.10.2. **Scores basés sur l'information mutuelle**<br/>([Mutual Information based scores](https://scikit-learn.org/stable/modules/clustering.html#mutual-information-based-scores))

Compte tenu de la connaissance des affectations de classe de vérité terrain `labels_true` et de nos affectations d'algorithme de regroupement des mêmes échantillons `labels_pred`, l'**information mutuelle** est une fonction qui mesure l'**accord** des deux affectations, en ignorant les permutations. Deux versions normalisées différentes de cette mesure sont disponibles, l'**information mutuelle normalisée (NMI)** et l'**information mutuelle ajustée (AMI)**. NMI est souvent utilisé dans la littérature, tandis que AMI a été proposé plus récemment et est **normalisé contre le hasard** :

In [11]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]

metrics.adjusted_mutual_info_score(labels_true, labels_pred)
# 0.22504...

0.2987924581708901

On peut permuter 0 et 1 dans les étiquettes prédites, renommer 2 en 3 et obtenir le même score :

In [12]:
labels_pred = [1, 1, 0, 0, 3, 3]
metrics.adjusted_mutual_info_score(labels_true, labels_pred)  
# 0.22504...

0.2987924581708901

[**`mutual_info_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mutual_info_score.html), [**`adjusted_mutual_info_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_mutual_info_score.html) et [**`normalized_mutual_info_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html) sont tous symétriques : échanger l'argument ne change pas le score. Ainsi, ils peuvent être utilisés comme **mesure consensuelle** :

In [13]:
metrics.adjusted_mutual_info_score(labels_pred, labels_true)  
# 0.22504...

0.2987924581708903

L'étiquetage parfait est noté 1,0 :

In [14]:
labels_pred = labels_true[:]
metrics.adjusted_mutual_info_score(labels_true, labels_pred)  
# 1.0

metrics.normalized_mutual_info_score(labels_true, labels_pred)  
# 1.0

1.0

Ce n'est pas vrai pour `mutual_info_score`, qui est donc plus difficile à juger :

In [16]:
metrics.mutual_info_score(labels_true, labels_pred)  
# 0.69...

0.693147180559945

Les mauvais (par exemple, les étiquetages indépendants) ont des scores non positifs :

In [15]:
labels_true = [0, 1, 2, 0, 3, 4, 5, 1]
labels_pred = [1, 1, 0, 0, 2, 2, 2, 2]
metrics.adjusted_mutual_info_score(labels_true, labels_pred)  
# -0.10526...

-0.16666666666666655

### Avantages

* **Les attributions d'étiquettes aléatoires (uniformes) ont un score AMI proche de 0,0** pour toute valeur de `n_clusters` et `n_samples` (ce qui n'est pas le cas pour l'information mutuelle brute ou la V-mesure par exemple).

* **Limite supérieure de 1** : les valeurs proches de zéro indiquent deux attributions d'étiquettes largement indépendantes, tandis que les valeurs proches de un indiquent un accord significatif. De plus, un AMI d'exactement 1 indique que les deux attributions d'étiquettes sont égales (avec ou sans permutation).

### Inconvénients

* Contrairement à l'inertie, **les mesures basées sur l'IM nécessitent la connaissance des classes de vérité terrain** alors qu'elles ne sont presque jamais disponibles dans la pratique ou nécessitent une affectation manuelle par des annotateurs humains (comme dans le cadre de l'apprentissage supervisé).<br/>Cependant, les mesures basées sur l'IM peuvent également être utiles dans un cadre purement non supervisé en tant que bloc de construction pour un indice de consensus qui peut être utilisé pour la sélection de modèles de regroupement.
* NMI et MI ne sont pas ajustés au hasard.

### Exemples

#### [**Ajustement pour le hasard dans l'évaluation des performances de clustering**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_adjusted_for_chance_measures.ipynb)<br/>([*Adjustment for chance in clustering performance evaluation*](https://scikit-learn.org/stable/auto_examples/cluster/plot_adjusted_for_chance_measures.html))

Analyse de l'impact de la taille de l'ensemble de données sur la valeur des mesures de regroupement pour les affectations aléatoires. Cet exemple inclut également l'indice Rand ajusté.

### Formulation mathématique

Supposons deux affectations d'étiquettes (des mêmes $N$ objets), $U$ et $V$. Leur entropie est la quantité d'incertitude pour un ensemble de partitions, définie par :

$$H(U) = - \sum_{i=1}^{|U|}P(i)\log(P(i))$$

où $P(i) = |U_i| / N$ est la probabilité qu'un objet choisi au hasard dans $U$ tombe dans la classe $U_i$. De même pour $V$ :

$$H(V) = - \sum_{j=1}^{|V|}P'(j)\log(P'(j))$$

Avec $P'(j) = |V_j| / N$. L'information mutuelle (MI) entre $U$ et $V$ est calculée par :

$$\text{MI}(U, V) = \sum_{i=1}^{|U|}\sum_{j=1}^{|V|}P(i, j)\log\left(\frac{P(i,j)}{P(i)P'(j)}\right)$$

où $P(i, j) = |U_i \cap V_j| / N$ est la probabilité qu'un objet pris au hasard tombe dans les deux classes $U_i$ et $V_j$

Il peut également être exprimé en formulation de cardinalité définie :

$$\text{MI}(U, V) = \sum_{i=1}^{|U|} \sum_{j=1}^{|V|} \frac{|U_i \cap V_j|}{N}\log\left(\frac{N|U_i \cap V_j|}{|U_i||V_j|}\right)$$

L'information mutuelle normalisée est définie comme

$$\text{NMI}(U, V) = \frac{\text{MI}(U, V)}{\text{mean}(H(U), H(V))}$$

Cette valeur de l'information mutuelle et également la variante normalisée n'est pas ajustée au hasard et aura tendance à augmenter à mesure que le nombre d'étiquettes différentes (grappes) augmente, quelle que soit la quantité réelle «d'informations mutuelles» entre les attributions d'étiquettes.

La valeur attendue de l'information mutuelle peut être calculée à l'aide de l'équation suivante [VEB2009]. Dans cette équation, $a_i = |U_i|$ (le nombre d'éléments dans $U_i$) et $b_j = |V_j|$ (le nombre d'éléments dans $V_j$).

$$E[\text{MI}(U,V)]=\sum_{i=1}^{|U|} \sum_{j=1}^{|V|} \sum_{n_{ij}=(a_i+b_j-N)^+
}^{\min(a_i, b_j)} \frac{n_{ij}}{N}\log \left( \frac{ N.n_{ij}}{a_i b_j}\right)
\frac{a_i!b_j!(N-a_i)!(N-b_j)!}{N!n_{ij}!(a_i-n_{ij})!(b_j-n_{ij})!
(N-a_i-b_j+n_{ij})!}$$

A partir de la valeur attendue, l'information mutuelle ajustée peut alors être calculée selon une forme similaire à celle de l'indice Rand ajusté :

$$\text{AMI} = \frac{\text{MI} - E[\text{MI}]}{\text{mean}(H(U), H(V)) - E[\text{MI}]}$$

Pour les informations mutuelles normalisées et les informations mutuelles ajustées, la valeur de normalisation est généralement une moyenne généralisée des entropies de chaque regroupement. Divers moyens généralisés existent, et aucune règle ferme n'existe pour en préférer un aux autres. La décision est en grande partie une base champ par champ; par exemple, dans la détection communautaire, la moyenne arithmétique est la plus courante. Chaque méthode de normalisation fournit des « comportements qualitativement similaires » [YAT2016]. Dans notre implémentation, ceci est contrôlé par le paramètre `average_method`.

Vinh et al. (2010) ont nommé des variantes de NMI et AMI par leur méthode de moyenne [VEB2010]. Leurs moyennes « sqrt » et « sum » sont les moyennes géométriques et arithmétiques ; nous utilisons ces noms plus largement communs.

### Références

[?] Strehl, Alexander, and Joydeep Ghosh (2002). [“**Cluster ensembles – a knowledge reuse framework for combining multiple partitions**](https://dl.acm.org/doi/pdf/10.1162/153244303321897735)[”](https://drive.google.com/file/d/1BQxDsFWFFabMlAwrmDwjFaEqIiiTMbVA/view?usp=share_link). Journal of Machine Learning Research 3: 583–617. doi:10.1162/153244303321897735.

[Wikipedia entry for the **(normalized) Mutual Information**](https://en.wikipedia.org/wiki/Mutual_information)

[Wikipedia entry for the **Adjusted Mutual Information**](https://en.wikipedia.org/wiki/Adjusted_mutual_information)

[VEB2009] Vinh, Epps, and Bailey, (2009). [“**Information theoretic measures for clusterings comparison**](https://dl.acm.org/doi/10.1145/1553374.1553511)[”](https://drive.google.com/file/d/1TWusEcciRGg1XklzyM8P8jKVr0JwCUYf/view?usp=share_link). Proceedings of the 26th Annual International Conference on Machine Learning - ICML ‘09. doi:10.1145/1553374.1553511. ISBN 9781605585161.

[VEB2010] Vinh, Epps, and Bailey, (2010). [“**Information Theoretic Measures for Clusterings Comparison: Variants, Properties, Normalization and Correction for Chance**](https://jmlr.csail.mit.edu/papers/volume11/vinh10a/vinh10a.pdf)[”](https://drive.google.com/file/d/144opdgyDLoPavOrxlMO9eh8e5TIi1hCV/view?usp=share_link). JMLR

[YAT2016] Yang, Algesheimer, and Tessone, (2016). [“**A comparative analysis of community detection algorithms on artificial networks**](https://zora.uzh.ch/id/eprint/127494/1/srep30750.pdf)[”](https://drive.google.com/file/d/1Ckhyhx2Fx0lk4i94dYOwdSMnqf_hADM1/view?usp=share_link). Scientific Reports 6: 30750. doi:10.1038/srep30750.


## 2.3.10.3. **Homogénéité, complétude et V-mesure**<br/>([Homogeneity, completeness and V-measure](https://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure)) 

Compte tenu de la connaissance des affectations de classe de vérité terrain des échantillons, il est possible de définir une métrique intuitive en utilisant l'analyse d'entropie conditionnelle.

En particulier, Rosenberg et Hirschberg (2007) définissent les deux objectifs souhaitables suivants pour toute affectation de cluster :
* **homogénéité** : chaque cluster ne contient que des membres d'une seule classe.
* **complétude** : tous les membres d'une classe donnée sont affectés au même cluster.

Nous pouvons transformer ces concepts en scores [**`homogeneity_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html) et [**`completeness_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html). Les deux sont délimités en dessous par 0,0 et au-dessus par 1,0 (plus c'est haut, mieux c'est) :

In [ ]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]

metrics.homogeneity_score(labels_true, labels_pred)
# 0.66...

metrics.completeness_score(labels_true, labels_pred)
# 0.42...

Leur moyenne harmonique appelée **V-measure** est calculée par [**`v_measure_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.v_measure_score.html) :

In [18]:
metrics.v_measure_score(labels_true, labels_pred)
# 0.51...

0.4999999999999998

La formule de cette fonction est la suivante :

$$v = \frac{(1 + \beta) \times \text{homogeneity} \times \text{completeness}}{(\beta \times \text{homogeneity} + \text{completeness})}$$

`beta` prend par défaut la valeur 1.0, mais pour utiliser une valeur inférieure à 1 pour beta :

In [17]:
metrics.v_measure_score(labels_true, labels_pred, beta=0.6)
# 0.54...

0.4705882352941175

plus de poids sera attribué à l'exhaustivité.

La V-mesure est en fait équivalente à l'information mutuelle (NMI) discutée ci-dessus, la fonction d'agrégation étant la moyenne arithmétique [B2011].

L'homogénéité, l'exhaustivité et la V-mesure peuvent être calculées en une fois en utilisant [**`homogeneity_completeness_v_measure`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html) comme suit :

In [ ]:
metrics.homogeneity_completeness_v_measure(labels_true, labels_pred)
# (0.66..., 0.42..., 0.51...)

L'affectation de clustering suivante est légèrement meilleure, car elle est homogène mais pas complète :

In [ ]:
labels_pred = [0, 0, 0, 1, 2, 2]
metrics.homogeneity_completeness_v_measure(labels_true, labels_pred)
# (1.0, 0.68..., 0.81...)

**Note** : [**`v_measure_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.v_measure_score.html) est **symétrique** : il peut être utilisé pour évaluer l'accord de deux affectations indépendantes sur le même jeu de données.

Ce n'est pas le cas pour [**`completeness_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html) et [**`homogeneity_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html) : tous deux sont liés par la relation :

    homogeneity_score(a, b) == completeness_score(b, a)

### Avantages

* **Scores limités** : 0,0 est aussi mauvais que possible, 1,0 est un score parfait.
* Interprétation intuitive : le regroupement avec une mauvaise V-mesure peut être **analysé qualitativement en termes d'homogénéité et d'exhaustivité** pour mieux sentir quel « type » d'erreurs est commis par l'affectation.
* **Aucune hypothèse n'est faite sur la structure du cluster** : peut être utilisé pour comparer des algorithmes de clustering tels que k-means qui suppose des formes de blobs isotropes avec les résultats d'algorithmes de clustering spectraux qui peuvent trouver des clusters avec des formes "pliées".

### Inconvénients

* Les métriques introduites précédemment ne **sont pas normalisées en ce qui concerne l'étiquetage aléatoire** : cela signifie qu'en fonction du nombre d'échantillons, de grappes et de classes de vérité terrain, un étiquetage complètement aléatoire ne donnera pas toujours les mêmes valeurs d'homogénéité, d'exhaustivité et donc de v-mesure. En particulier, l'**étiquetage aléatoire ne donnera pas de scores nuls, surtout lorsque le nombre de grappes est important**.<br/>Ce problème peut être ignoré en toute sécurité lorsque le nombre d'échantillons est supérieur à mille et que le nombre de grappes est inférieur à 10. **Pour des tailles d'échantillon plus petites ou un plus grand nombre de grappes, il est plus sûr d'utiliser un indice ajusté tel que l'indice Rand ajusté (ARI)**.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_adjusted_for_chance_measures_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_adjusted_for_chance_measures_001.png" style="width: 640.0px; height: 480.0px;" />

* Ces métriques **nécessitent la connaissance des classes de vérité terrain** alors qu'elles ne sont presque jamais disponibles dans la pratique ou nécessitent une affectation manuelle par des annotateurs humains (comme dans le cadre de l'apprentissage supervisé).

### Exemples

#### [**Ajustement pour le hasard dans l'évaluation des performances de clustering**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_adjusted_for_chance_measures.ipynb)<br/>([*Adjustment for chance in clustering performance evaluation*](https://scikit-learn.org/stable/auto_examples/cluster/plot_adjusted_for_chance_measures.html))

Analyse de l'impact de la taille de l'ensemble de données sur la valeur des mesures de regroupement pour les affectations aléatoires.

### Formulation mathématique

Les scores d'homogénéité et de complétude sont formellement donnés par :

$$h = 1 - \frac{H(C|K)}{H(C)}$$

$$c = 1 - \frac{H(K|C)}{H(K)}$$

où $H(C|K)$ est l'**entropie conditionnelle des classes compte tenu des affectations de cluster** et est donnée par :

$$H(C|K) = - \sum_{c=1}^{|C|} \sum_{k=1}^{|K|} \frac{n_{c,k}}{n}
\cdot \log\left(\frac{n_{c,k}}{n_k}\right)$$

et $H(C)$ est l'**entropie des classes** et est donnée par :

$$H(C) = - \sum_{c=1}^{|C|} \frac{n_c}{n} \cdot \log\left(\frac{n_c}{n}\right)$$

avec $n$ le nombre total d'échantillons, $n_c$ et $n_k$ le nombre d'échantillons appartenant respectivement à la classe $c$ et à la grappe $k$, et enfin $n_{c, k}$ le nombre d'échantillons de la classe $c$ affectée à la grappe $k$.

L'**entropie conditionnelle des clusters d'une classe donnée** $H(K|C)$ et l'**entropie des clusters** $H(K)$ sont définis de manière symétrique.

Rosenberg et Hirschberg définissent en outre la **V-mesure** comme la **moyenne harmonique de l'homogénéité et de l'exhaustivité** :

$$v = 2 \cdot \frac{h \cdot c}{h + c}$$

### Références

[?] [“**V-Measure: A conditional entropy-based external cluster evaluation measure**](https://aclanthology.org/D07-1043.pdf)[”](https://drive.google.com/file/d/1ycDJJmQqTMHp3LCR6iI1c8QjqXmYtBQn/view?usp=share_link) Andrew Rosenberg and Julia Hirschberg, 2007

[B2011] [“**Identication and Characterization of Events in Social Media**](http://www.cs.columbia.edu/~hila/hila-thesis-distributed.pdf)[”](https://drive.google.com/file/d/1Sbt6Qu-zE_wPaEBVefL6hLb4krGyuFr-/view?usp=share_link), Hila Becker, PhD Thesis.

## 2.3.10.4. Scores de Fowlkes-Mallows

L'indice Fowlkes-Mallows ([**`sklearn.metrics.fowlkes_mallows_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fowlkes_mallows_score.html)) peut être utilisé lorsque les affectations de classe de vérité terrain des échantillons sont connues. Le score Fowlkes-Mallows FMI est défini comme la moyenne géométrique de la précision et du rappel par paire :

$$\text{FMI} = \frac{\text{TP}}{\sqrt{(\text{TP} + \text{FP}) (\text{TP} + \text{FN})}}$$

Où `TP` est le nombre de **vrais positifs** (c'est-à-dire le nombre de paires de points qui appartiennent aux mêmes clusters dans les étiquettes vraies et les étiquettes prédites), `FP` est le nombre de **faux positifs** (c'est-à-dire le nombre de paires de points qui appartiennent aux mêmes clusters dans les vraies étiquettes et non dans les étiquettes prédites) et FN est le nombre de **faux négatifs** (c'est-à-dire le nombre de paires de points qui appartiennent aux mêmes grappes dans les étiquettes prédites et non dans les vraies étiquettes).

Le score varie de 0 à 1. Une valeur élevée indique une bonne similarité entre deux clusters.

In [19]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]
metrics.fowlkes_mallows_score(labels_true, labels_pred)
# 0.47140...

0.4714045207910317

On peut permuter 0 et 1 dans les étiquettes prédites, renommer 2 en 3 et obtenir le même score :

In [20]:
labels_pred = [1, 1, 0, 0, 3, 3]
metrics.fowlkes_mallows_score(labels_true, labels_pred)
# 0.47140...

0.4714045207910317

L'étiquetage parfait est noté 1,0

In [21]:
labels_pred = labels_true[:]
metrics.fowlkes_mallows_score(labels_true, labels_pred)
# 1.0

1.0

Les mauvais (par exemple, les étiquetages indépendants) ont des scores nuls :

In [22]:
labels_true = [0, 1, 2, 0, 3, 4, 5, 1]
labels_pred = [1, 1, 0, 0, 2, 2, 2, 2]
metrics.fowlkes_mallows_score(labels_true, labels_pred)
0.0

0.0

### Avantages

* **Les attributions d'étiquettes aléatoires (uniformes) ont un score FMI proche de 0,0** pour toute valeur de `n_clusters` et `n_samples` (ce qui n'est pas le cas pour l'information mutuelle brute ou la V-mesure par exemple).
* **Borne supérieure à 1**: les valeurs proches de zéro indiquent deux attributions d'étiquettes largement indépendantes, tandis que les valeurs proches de un indiquent un accord significatif. De plus, des valeurs d'exactement 0 indiquent des attributions d'étiquettes **purement** indépendantes et un FMI d'exactement 1 indique que les deux attributions d'étiquettes sont égales (avec ou sans permutation).
* **Aucune hypothèse n'est faite sur la structure du cluster** : peut être utilisé pour comparer des algorithmes de clustering tels que k-means qui suppose des formes de blobs isotropes avec les résultats d'algorithmes de clustering spectraux qui peuvent trouver des clusters avec des formes "pliées".

### Inconvénients

* Contrairement à l'inertie, **les mesures basées sur FMI nécessitent la connaissance des classes de vérité terrain** alors qu'elles ne sont presque jamais disponibles dans la pratique ou nécessitent une affectation manuelle par des annotateurs humains (comme dans le cadre de l'apprentissage supervisé).

### Références

E. B. Fowkles and C. L. Mallows, 1983. [“**A method for comparing two hierarchical clusterings**](https://www.semanticscholar.org/paper/A-Method-for-Comparing-Two-Hierarchical-Clusterings-Fowlkes-Mallows/ededd54b4f7578802ce81fe7a34c05d93e5e09ee)[”](https://drive.google.com/file/d/1-KzvRHfFIDsQ_zyPUhO3uXhUJuLjDJrL/view?usp=share_link). Journal of the American Statistical Association.

[Wikipedia entry for the **Fowlkes-Mallows Index**](https://en.wikipedia.org/wiki/Fowlkes%E2%80%93Mallows_index)


## 2.3.10.5. Coefficient de silhouette

Si les étiquettes de vérité terrain ne sont pas connues, l'évaluation doit être effectuée à l'aide du modèle lui-même. Le coefficient de silhouette ([**`sklearn.metrics.silhouette_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)) est un exemple d'une telle évaluation, où un score de coefficient de silhouette plus élevé se rapporte à un modèle avec des clusters mieux définis. Le Coefficient Silhouette est défini pour chaque échantillon et est composé de deux scores :
* **a** : La distance moyenne entre un échantillon et tous les autres points de la même classe.
* **b** : La distance moyenne entre un échantillon et tous les autres points de la grappe la plus proche.

Le coefficient de silhouette $s$ pour un seul échantillon est alors donné par :

$$s = \frac{b - a}{max(a, b)}$$

Le coefficient de silhouette pour un ensemble d'échantillons est donné comme la moyenne du coefficient de silhouette pour chaque échantillon.

In [23]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

En utilisation normale, le coefficient de silhouette est appliqué aux résultats d'une analyse de cluster.

In [24]:
import numpy as np
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_
metrics.silhouette_score(X, labels, metric='euclidean')
# 0.55...

c:\Users\franc\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


0.5528190123564095

### Avantages

* Le score est compris entre -1 pour un clustering incorrect et +1 pour un clustering très dense. Les scores autour de zéro indiquent des clusters qui se chevauchent.
* Le score est plus élevé lorsque les clusters sont denses et bien séparés, ce qui correspond à un concept standard de cluster.

### Inconvénients

* Le coefficient de silhouette est généralement plus élevé pour les clusters convexes que pour les autres concepts de clusters, tels que les clusters basés sur la densité comme ceux obtenus via DBSCAN.

### Exemples

#### [**Sélection du nombre de clusters avec analyse de silhouette sur KMeans clustering**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_kmeans_silhouette_analysis.ipynb)<br/>([*Selecting the number of clusters with silhouette analysis on KMeans clustering*](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html))

Dans cet exemple l'analyse de silhouette est utilisée pour choisir une valeur optimale pour `n_clusters`.

### Références

Peter J. Rousseeuw (1987). [“**Silhouettes: a Graphical Aid to the Interpretation and Validation of Cluster Analysis**](https://www.sciencedirect.com/science/article/pii/0377042787901257?via%3Dihub)[”](https://drive.google.com/file/d/1T-cjvWWtWF_gUIixc3Rcmmq4wOLYhWvQ/view?usp=share_link). Computational and Applied Mathematics 20: 53–65.





## 2.3.10.6. Indice de Calinski-Harabasz

Si les étiquettes de vérité de terrain ne sont pas connues, l'indice Calinski-Harabasz ([**`sklearn.metrics.calinski_harabasz_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html)) - également connu sous le nom de critère du rapport de variance - peut être utilisé pour évaluer le modèle, où un score Calinski-Harabasz plus élevé se rapporte à un modèle avec clusters mieux définis.

L'indice est le rapport de la somme de la dispersion inter-clusters et de la dispersion intra-cluster pour tous les clusters (où la dispersion est définie comme la somme des distances au carré) :

In [25]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

En usage normal, l'indice de Calinski-Harabasz est appliqué aux résultats d'une analyse de cluster :

In [26]:
import numpy as np
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_
metrics.calinski_harabasz_score(X, labels)
# 561.62...

c:\Users\franc\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


561.62775662962

### Avantages

* Le score est plus élevé lorsque les clusters sont denses et bien séparés, ce qui correspond à un concept standard de cluster.

* Le score est rapide à calculer.

### Inconvénients

* L'indice Calinski-Harabasz est généralement plus élevé pour les clusters convexes que pour les autres concepts de clusters, tels que les clusters basés sur la densité comme ceux obtenus via DBSCAN.

### Formulation mathématique

Pour un ensemble de données $E$ de taille $n_E$ qui a été regroupée en $k$ grappes, le score de Calinski-Harabasz $s$ est défini comme le rapport de la moyenne de dispersion inter-grappes et de la dispersion intra-grappe :

$$s = \frac{\mathrm{tr}(B_k)}{\mathrm{tr}(W_k)} \times \frac{n_E - k}{k - 1}$$

où $\mathrm{tr}(B_k)$ est la trace de la matrice de dispersion inter-groupes et $\mathrm{tr}(W_k)$ est la trace de la matrice de dispersion intra-grappe définie par :

$$W_k = \sum_{q=1}^k \sum_{x \in C_q} (x - c_q) (x - c_q)^\top$$
$$B_k = \sum_{q=1}^k n_q (c_q - c_E) (c_q - c_E)^\top$$

avec $C_q$ l'ensemble des points dans le cluster $q$, $c_q$ le centre du cluster $q$, $c_E$ le centre de $E$, et $n_q$ le nombre de points dans le cluster $q$.

### Références

[?] Caliński, T., & Harabasz, J. (1974). [“**A Dendrite Method for Cluster Analysis**](https://www.researchgate.net/publication/233096619_A_Dendrite_Method_for_Cluster_Analysis)[”](https://drive.google.com/file/d/1wVMKVAYxRrM4Ma9EwqJ5xqv3a_L0hvQT/view?usp=share_link). Communications in Statistics-theory and Methods 3: 1-27.


## 2.3.10.7. Davies-Bouldin Index

If the ground truth labels are not known, the Davies-Bouldin index ([**`sklearn.metrics.davies_bouldin_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html)) can be used to evaluate the model, where a lower Davies-Bouldin index relates to a model with better separation between the clusters.

This index signifies the average ‘similarity’ between clusters, where the similarity is a measure that compares the distance between clusters with the size of the clusters themselves.

Zero is the lowest possible score. Values closer to zero indicate a better partition.

In normal usage, the Davies-Bouldin index is applied to the results of a cluster analysis as follows:

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
kmeans = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans.labels_
davies_bouldin_score(X, labels)
# 0.6619...

### Avantages

* Le calcul de Davies-Bouldin est plus simple que celui des scores Silhouette.
* L'indice est uniquement basé sur des quantités et des caractéristiques inhérentes à l'ensemble de données car son calcul n'utilise que des distances ponctuelles.

### Inconvénients

* L'indice Davies-Boulding est généralement plus élevé pour les clusters convexes que pour les autres concepts de clusters, tels que les clusters basés sur la densité comme ceux obtenus à partir de DBSCAN.
* L'utilisation de la distance centroïde limite la distance métrique à l'espace euclidien.

### Formulation mathématique

L'indice est défini comme la similarité moyenne entre chaque cluster $C_i$ pour $i = 1, \cdots, k$ et son $C_j$ le plus similaire. Dans le cadre de cet indice, la similarité est définie comme une mesure $R_{ij}$ qui équilibre :
* $s_i$, la distance moyenne entre chaque point du cluster $i$ et le centroïde de ce cluster - également connu sous le nom de diamètre de cluster.
* $d_{ij}$, la distance entre les centres de gravité des clusters $i$ et $j$.

Un choix simple pour construire $R_{ij}$ de telle sorte qu'il soit non négatif et symétrique est :

$$R_{ij} = \frac{s_i + s_j}{d_{ij}}$$

Alors l'indice de Davies-Bouldin est défini comme :

$$DB = \frac{1}{k} \sum_{i=1}^k \max_{i \neq j} R_{ij}$$

### Références

[?] Davies, David L.; Bouldin, Dld W. (1979). [“**A Cluster Separation Measure**](https://ieeexplore.ieee.org/document/4766909)[”](https://drive.google.com/file/d/1UYs_tDk5azMulhvMTpKhj5Agw930cfW_/view?usp=share_link) IEEE Transactions on Pattern Analysis and Machine Intelligence. PAMI-1 (2): 224-227.

[?] Halkidi, Maria; Batistakis, Yannis; Vazirgiannis, Michalis (2001). [“**On Clustering Validation Techniques**](https://link.springer.com/article/10.1023/A:1012801612483)[”](https://drive.google.com/file/d/1Ku1fxm7MOm5n8ImQ3UoUbN_F84zgjz_8/view?usp=share_link) Journal of Intelligent Information Systems, 17(2-3), 107-145.

[?] [Wikipedia entry for **Davies-Bouldin index**](https://en.wikipedia.org/wiki/Davies%E2%80%93Bouldin_index).


### 2.3.10.8. Matrice de contingence

La matrice de contingence ([**`sklearn.metrics.cluster.contingency_matrix`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cluster.contingency_matrix.html)) indique la cardinalité d'intersection pour chaque paire de cluster vraie/prédite. La matrice de contingence fournit des statistiques suffisantes pour toutes les métriques de clustering où les échantillons sont indépendants et distribués de manière identique et il n'est pas nécessaire de tenir compte du fait que certaines instances ne sont pas regroupées.

Voici un exemple:

In [27]:
from sklearn.metrics.cluster import contingency_matrix
x = ["a", "a", "a", "b", "b", "b"]
y = [0, 0, 1, 1, 2, 2]
contingency_matrix(x, y)
# array([[2, 1, 0],
#       [0, 1, 2]])

array([[2, 1, 0],
       [0, 1, 2]], dtype=int64)

La première ligne du tableau de sortie indique qu'il y a trois échantillons dont le vrai cluster est "a". Parmi eux, deux sont dans le cluster prédit 0, un est dans 1 et aucun n'est dans 2. Et la deuxième ligne indique qu'il y a trois échantillons dont le vrai cluster est "b". Parmi eux, aucun n'est dans le cluster prédit 0, un est dans 1 et deux sont dans 2.

Une [**matrice de confusion** (3.3.2.6)](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) pour la classification est une matrice de contingence carrée où l'ordre des lignes et des colonnes correspond à une liste de classes.

### Avantages

* Permet d'examiner la propagation de chaque vrai cluster à travers les clusters prédits et vice versa.
* Le tableau de contingence calculé est généralement utilisé dans le calcul d'une statistique de similarité (comme les autres répertoriées dans ce document) entre les deux regroupements.

### Inconvénients

* La matrice de contingence est facile à interpréter pour un petit nombre de clusters, mais devient très difficile à interpréter pour un grand nombre de clusters.
* Il ne donne pas une seule métrique à utiliser comme objectif pour l'optimisation du clustering.

### Références

[Wikipedia entry for **contingency matrix**](https://en.wikipedia.org/wiki/Contingency_table)

## 2.3.10.9. Matrice de confusion des paires

La matrice de confusion des paires ([**`sklearn.metrics.cluster.pair_confusion_matrix`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cluster.pair_confusion_matrix.html)) est une matrice de similarité 2x2

$$\begin{split}C = \left[\begin{matrix}
C_{00} & C_{01} \\
C_{10} & C_{11}
\end{matrix}\right]\end{split}$$

entre deux clusterings calculés en considérant toutes les paires d'échantillons et en comptant les paires qui sont affectées dans le même cluster ou dans des clusters différents sous les clusterings vrais et prédits.

Il contient les entrées suivantes :

$C_{00}$ : nombre de paires avec les deux regroupements ayant les échantillons non regroupés

$C_{10}$ : nombre de paires avec le véritable regroupement d'étiquettes ayant les échantillons regroupés mais l'autre regroupement n'ayant pas les échantillons regroupés

$C_{01}$ : nombre de paires avec le vrai regroupement d'étiquettes n'ayant pas les échantillons regroupés mais l'autre regroupement ayant les échantillons regroupés

$C_{11}$ : nombre de paires avec les deux regroupements ayant les échantillons regroupés

Considérant une paire d'échantillons regroupés en une paire positive, alors, comme dans la classification binaire, le nombre de vrais négatifs est $C_{00}$, les faux négatifs sont $C_{10}$, les vrais positifs sont $C_{11}$ et les faux positifs sont $C_{01}$.

Les libellés qui correspondent parfaitement ont tous des entrées non nulles sur la diagonale, quelles que soient les valeurs réelles des libellés :

In [28]:
from sklearn.metrics.cluster import pair_confusion_matrix
pair_confusion_matrix([0, 0, 1, 1], [0, 0, 1, 1])
# array([[8, 0],
#        [0, 4]])
pair_confusion_matrix([0, 0, 1, 1], [1, 1, 0, 0])
# array([[8, 0],
#        [0, 4]])

array([[8, 0],
       [0, 4]], dtype=int64)

Les étiquetages qui affectent tous les membres de classes aux mêmes clusters sont complets mais peuvent ne pas toujours être purs, donc pénalisés, et ont des entrées non nulles hors diagonale :

In [29]:
pair_confusion_matrix([0, 0, 1, 2], [0, 0, 1, 1])
# array([[8, 2],
#        [0, 2]])

array([[8, 2],
       [0, 2]], dtype=int64)

La matrice n'est pas symétrique

In [30]:
pair_confusion_matrix([0, 0, 1, 1], [0, 0, 1, 2])
# array([[8, 0],
#        [2, 2]])

array([[8, 0],
       [2, 2]], dtype=int64)

Si les membres des classes sont complètement répartis sur différents clusters, l'affectation est totalement incomplète, donc la matrice a toutes les entrées en diagonale nulle :

In [31]:
pair_confusion_matrix([0, 0, 0, 0], [0, 1, 2, 3])
# array([[ 0,  0],
#        [12,  0]])

array([[ 0,  0],
       [12,  0]], dtype=int64)

### Références

[?] L. Hubert and P. Arabie, “**Comparing Partitions**”, Journal of Classification, 1985[.](https://drive.google.com/file/d/1tLtEWnFFD6PZ_uv8uwI_UNTBvZQTa0og/view?usp=share_link)